# Molecular Quantum Mechanics (CB2070)
## SOLUTION Computer lab 4: Group theory and spatial symmetry of wave functions
---
Name: J.H. Andersen with stolen content from VT24 and VT25 student submissions

Date: August 2025

---

In [ ]:
import veloxchem as vlx
import numpy as np

In [ ]:
np.set_printoptions(precision=4, suppress=True, linewidth=132) # printout format of NumPy arrays

### 1. SCF optimization

In [ ]:
mol_str = """
H        1.21655197    0.92414474    0.00000000
H        1.21655197   -0.92414474    0.00000000
H       -1.21655197   -0.92414474    0.00000000
H       -1.21655197    0.92414474    0.00000000
C        0.67759997    0.00000000    0.00000000
C       -0.67759997    0.00000000    0.00000000
"""
molecule = vlx.Molecule.read_str(mol_str, units='angstrom')

In [ ]:
scf_drv = vlx.ScfRestrictedDriver()
basis = vlx.MolecularBasis.read(molecule, "sto-3g")

scf_results = scf_drv.compute(molecule, basis)

#### Identify the occupied and unoccupied orbitals

In [ ]:
nocc = molecule.number_of_alpha_electrons()
nvir = basis.get_dimensions_of_basis() - nocc
eocc = scf_results['E_alpha'][:nocc]

In [ ]:
print(f"Number of occupied orbitals: {nocc}")
print(f"Number of unoccupied orbitals: {nvir}")
print()
print(f"Energies of occupied MOs: {eocc}")

There are 14 MOs in total, out of which 8 are occupied and 6 are unoccupied (virtual)

### 2. Determine the irrep of each MO

**FYI:** The ordering of atomic orbitals in the Veloxchem output is

$\chi_{1s}^{H1}$, $\chi_{1s}^{H2}$, $\chi_{1s}^{H3}$, $\chi_{1s}^{H4}$
$\chi_{1s}^{C1}$, $\chi_{2s}^{C1}$, $\chi_{1s}^{C2}$, $\chi_{2s}^{C2}$
$\chi_{2py}^{C1}$, $\chi_{2py}^{C2}$, $\chi_{2pz}^{C1}$, $\chi_{2pz}^{C2}$, $\chi_{2px}^{C1}$, $\chi_{2px}^{C2}$

and H1, H2, H3, H4 are numbered according to the order of the atoms in the molecular coordinates.

#### THE PROCEDURE IS AS FOLLOWS
1. Draw the C2H4 molecule in the z-plane and number the H and C atoms according to their occurence in the coordinates. You need to get this numbering right, or you will mess up a couple of the MO symmetries
2. In the MO coefficient array, the rows are the AOs (in the order listes above) and the columns are the MOs. Write down the linear combinations of the AOs for each MO noting only the sign of the coefficient and not its value. For example, the first MO is (**OBS: you might get the opposite signs when running the SCF on your laptop. It does not matter, as long as the relative signs agree**)
$$\phi_1 = -1s^{H_1} - 1s^{H_2} - 1s^{H_3} - 1s^{H_4} + 1s^{C_1} + 2s^{C_1}  + 1s^{C_2} + 2s^{C_2} + 2p_x^{C_1} + 2p_x^{C_2}$$
3. Go through each MO and note how the AO combination transform according to the symmetry operations of the $D_{2h}$ point group
4. Compare the signs of the symmetry operations to the character table of the point group and determine the irrep of each MO
5. Bravo, you did it! It only cost you your sanity and more hours than you are willing to admit :)

In [ ]:
# get the MO coefficients
C = scf_drv.scf_tensors['C_alpha']
# print
print('MO coefficients:\n', C)

#### List the MOs and their irrep

The symmetries of the MOs are

$A_g: \phi_1 / \phi_3 / \phi_6 / \phi_{11}$

$B_{1g}: \phi_7 / \phi_{14} $

$B_{2g}: \phi_9 $

$B_{1u}: \phi_8 $

$B_{2u}: \phi_5 / \phi_{10} $

$B_{3u}: \phi_2 / \phi_4 / \phi_{12} / \phi_{13}$

### 3. Create a unitary transformation matrix and produce SAOs

#### 3.a Define a unitary transformation matrix

In [ ]:
# define a unitary transformation matrix
# make sure that it fulfills *all* requirements specified in the exercise

# use the unitary matrix that was determined in the related exercise session

# TODO: define a unitary transformation matrix, and carry out the transformation
U_H_1s = 1/2 * np.array([[1,1,1,1],[1,1,-1,-1],[1,-1,-1,1],[1,-1,1,-1]])
U_C_1s = 1/np.sqrt(2) * np.array([[1,1],[1,-1]])
U_C_2s = 1/np.sqrt(2) * np.array([[1,1],[1,-1]])
U_C_2py = 1/np.sqrt(2) * np.array([[1,1],[1,-1]])
U_C_2pz = 1/np.sqrt(2) * np.array([[1,1],[1,-1]])
U_C_2px = 1/np.sqrt(2) * np.array([[1,1],[1,-1]])

nao = nocc + nvir
U = np.zeros((nao, nao))

U[0:4,0:4] = U_H_1s
U[4:6,4:6] = U_C_1s
U[6:8,6:8] = U_C_2s
U[8:10,8:10] = U_C_2py
U[10:12,10:12] = U_C_2pz
U[12:14,12:14] = U_C_2px

# note that row 5 and 6 are swapped compared to the organization of atomic orbitals made by VeloxChem. This is handled later on.
# they are rearranged according to: ...1s(C1)1s(C2)2s(C1)2s(C2)...

In [ ]:
print('Symmetry-adapted coefficients:\n', U)

#### 3.b Determine the symmetry of the SAOs

The transformation matrix should also accomplish an ordering of the SAOs according to the symmetry in the $D_{2h}$ character table. To accomplish this each, SAO (column) is symmetry labeled, then grouped and ordered ($A_g, B_{1g}, B_{2g}, B_{2u}, B_{3u}$). 

Symmetry labelling and ordeing:

$A_g: SAO_1, SAO_5, SAO_7, SAO_{14}$

$B_{1g}: SAO_4, SAO_{10}$ 

$B_{2g}: SAO_{12}$ 

$B_{1u}: SAO_{11}$

$B_{2u}: SAO_3, SAO_{9}$

$B_{3u}: SAO_2, SAO_6, SAO_8, SAO_{13}$

#### 3.c Reorganize the transformation matrix according to the irreps

In [ ]:
# reorganize U according to character table
U_tmp = np.array([U[:,0], U[:,4], U[:,6], U[:,13], U[:,3], U[:,9], U[:,11], U[:,10], U[:,2], U[:,8], U[:,1], U[:,5], U[:,7], U[:,12]]).T

# reorganize U so ordering of 1s and 2s on C1 and C2 match that of VeloxChem by swapping row 5 and 6
U_prime = U_tmp.copy()

U_prime[5,:] = U_tmp[6,:]
U_prime[6,:] = U_tmp[5,:]

In [ ]:
print('Reorganized unitary transformation matrix:\n', U_prime)

### 4. Determine the overlap matrix in the SAO basis

Complete the equation
$$ \mathbf{S'} = \mathbf{U}^{\dagger} \mathbf{S} \mathbf{U} $$

In [ ]:
# get the overlap matrix in the original basis
S = scf_drv.scf_tensors['S']

# transform the overlap matrix
S_prime = np.matmul(U_prime.T, np.matmul(S, U_prime))

In [ ]:
print('Overlap matrix in SAO basis:\n', S_prime)

#### Comment on the matrix structure

The transformed overlap matrix is block diagonal since orbitals belonging to different irreps are orthogonal.

The blocks consist of the SAOs of each symmetry, so the left upper corner consists of SAOs with $A_g$ symmetry. The second bock on the diagonal is the SAOs with $B_{1g}$ symmetry, then comes to 1 dimensional blocks, the first with $B_{2g}$ and the second with $B_{1u}$. The two last block are the 2 dimensianl $B_{2u}$ and the 4 dimensional $B_{3u}$. We clearly see that the elements of the overlap matrix are onbly nonzero between basis funcitons of the same symmetry.

### 5. Determine the MO coefficients in the new basis

Complete the equation
$$ \mathbf{C'} = \mathbf{U}^{\dagger} \mathbf{C} $$

In [ ]:
# determine C'
C_prime = np.matmul(U_prime.T, C)

In [ ]:
print('MO coefficients in SAO basis:\n', C_prime)

#### Comment on the matrix structure

The rows correspond to the different symmetry adapted atomic orbitals, and since SAOs of different irreps don't mix, $\mathbf{C}'$ is much more sparse than $\mathbf{C}$.
It is seen from the structure of the matrix that the MOs are consituted of SAOs of the same symmetry only.

### 6. Determine the dipole moment in the (original) MO basis

In [ ]:
# get the AO representation of the dipole moment
dipole_drv = vlx.ElectricDipoleIntegralsDriver()

dipole_mats = dipole_drv.compute(molecule, basis)

mu_x = dipole_mats.x_to_numpy()
mu_y = dipole_mats.y_to_numpy()
mu_z = dipole_mats.z_to_numpy()

# transform to the MO representation of the operator

mu_x_MO = np.matmul(C.T, np.matmul(mu_x, C))
mu_y_MO = np.matmul(C.T, np.matmul(mu_y, C))
mu_z_MO = np.matmul(C.T, np.matmul(mu_z, C))

In [ ]:
print('Dipole moment operator in MO basis, x-component:\n', mu_x_MO)
print('\nDipole moment operator in MO basis, y-component:\n', mu_y_MO)
print('\nDipole moment operator in MO basis, z-component:\n', mu_z_MO)

#### Discuss the results from a symmetry perspective

The row and column index each correspond to an MO. The MOs were symmetry labeled in task 2 and we know that 

$$ \langle \Psi_f |\mu_x| \Psi_i \rangle \neq 0$$

only if $$\Gamma(\Psi_f) \otimes \Gamma(\hat{\Omega}) \otimes\ \Gamma(\Psi_i) = \Gamma(\hat{I}) $$

Checking the irreps of the MOs as well as the irrep of the Cartesian coordinates, one can find the overall irrep of the matrix element by using the direct product table,

https://www.webqc.org/symmetrypointgroup-d2h.html

An example of a zero matrix element is:

$\Gamma(\phi_1) \otimes \Gamma(\mu_x) \otimes \Gamma(\phi_1) = A_g \otimes B_{3u} \otimes A_g = B_{3u} \neq A_g$: the element is zero.

$\Gamma(\phi_8) \otimes \Gamma(\mu_z) \otimes \Gamma(\phi_3) = B_{1u} \otimes B_{1u} \otimes Ag = A_g$: the element is non-zero.

### 7. Determine the dipole moment in the SAO basis

In [ ]:
# basis transformation of the dipole operator from its original AO representation
mu_x_SAO = np.matmul(U_prime.T , np.matmul(mu_x , U_prime))
mu_y_SAO = np.matmul(U_prime.T , np.matmul(mu_y , U_prime))
mu_z_SAO = np.matmul(U_prime.T , np.matmul(mu_z , U_prime))

In [ ]:
print('Dipole moment operator in SAO basis, x-component:\n', mu_x_SAO)
print('\nDipole moment operator in SAO basis, y-component:\n', mu_y_SAO)
print('\nDipole moment operator in SAO basis, z-component:\n', mu_z_SAO)

#### Comment on the matrix structure

In the new SAO basis, the rows and columns of the dipole moment matrices are organized by irreps: Ag (4), B1g (2), B2g (1), B1u (1), B2u (2) and B3u (4). Thus, only the blocks of the matrices where the symmetry product of the rows, the columns and the dipole moment cartesian component is Ag has non-zero values. For example the last four rows (B3u) and the first four columns (Ag) of the $x$-direction (B3u) gives the overall symmetry product Ag, and thus non-zero values are found in the block.

### 8. Determine the Cartesian components of transition dipole moment

In [ ]:
# determine the three transition dipole moment components between the HOMO and LUMO

# the HOMO is MO 8 (idx 7) and the LUMO is MO 9 (idx 8)
f_x = mu_x_MO[7,8]
f_y = mu_y_MO[7,8]
f_z = mu_z_MO[7,8]

In [ ]:
print(f'Dipole moment x-component: {f_x:2.4f}')
print(f'Dipole moment y-component: {f_y:2.4f}')
print(f'Dipole moment z-component: {f_z:2.4f}')

#### Discuss the results from a symmetry perspective

$\phi_8$ spans $B_{1u}$;

$\phi_9$ spans $B_{2g}$;

$\mu_x$ spans $B_{3u}$

$\Gamma(\phi_8) \otimes \Gamma(\mu_x) \otimes \Gamma(\phi_9) = B_{1u} \otimes B_{3u} \otimes B_{2g} = A_g$

This component is the only one where the symmetry product is $A_g$ and therefore, it is only the x-component of the dipole moment that contributes.

### 9. Determine the absorption intensity for the HOMO-LUMO transition

In [ ]:
# calculate intensity of the transition identified in (8)
I = f_x**2

In [ ]:
print(f"Absorption strength: {I:2.3f}")

## THE END